# Exploring Covid-19 Data

##### William Ho

### Import the data

First make sure to install necessary packages:

In [23]:
import sys
!{sys.executable} -m pip install kagglehub pandas matplotlib ipython-sql --upgrade

   ---------------------------------------- 0.0/44.4 kB ? eta -:--:--
   ---------------------------------------- 44.4/44.4 kB 1.1 MB/s eta 0:00:00


  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.

[notice] A new release of pip is available: 24.0 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


Then import the required libraries and data set:

In [13]:
import kagglehub

path = kagglehub.dataset_download("imdevskp/corona-virus-report")
print("Path to dataset files:", path)

c:\Users\willr\AppData\Local\Programs\Python\Python312\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


100%|██████████| 19.0M/19.0M [00:19<00:00, 1.00MB/s]

Extracting files...


Path to dataset files: C:\Users\willr\.cache\kagglehub\datasets\imdevskp\corona-virus-report\versions\166


In [14]:
import os

print("Files in dataset directory:")
print(os.listdir(path))

Files in dataset directory:
['country_wise_latest.csv', 'covid_19_clean_complete.csv', 'day_wise.csv', 'full_grouped.csv', 'usa_county_wise.csv', 'worldometer_data.csv']


In [ ]:
import pandas as pd

csv_path = os.path.join(path, "covid_19_clean_complete.csv")
df = pd.read_csv(csv_path)
df.head()

,Province/State,Country/Region,Lat,Long,Date,Confirmed,Deaths,Recovered,Active,WHO Region
0,NaN,Afghanistan,33.93911,67.709953,2020-01-22,0,0,0,0,Eastern Mediterranean
1,NaN,Albania,41.15330,20.168300,2020-01-22,0,0,0,0,Europe
2,NaN,Algeria,28.03390,1.659600,2020-01-22,0,0,0,0,Africa
3,NaN,Andorra,42.50630,1.521800,2020-01-22,0,0,0,0,Europe
4,NaN,Angola,-11.20270,17.873900,2020-01-22,0,0,0,0,Africa


Now we have an idea of what the data looks like, lets also look at all the countries included

In [ ]:
df['Country/Region'].unique()

array(['Afghanistan', 'Albania', 'Algeria', 'Andorra', 'Angola',
       'Antigua and Barbuda', 'Argentina', 'Armenia', 'Australia',
       'Austria', 'Azerbaijan', 'Bahamas', 'Bahrain', 'Bangladesh',
       'Barbados', 'Belarus', 'Belgium', 'Benin', 'Bhutan', 'Bolivia',
       'Bosnia and Herzegovina', 'Brazil', 'Brunei', 'Bulgaria',
       'Burkina Faso', 'Cabo Verde', 'Cambodia', 'Cameroon', 'Canada',
       'Central African Republic', 'Chad', 'Chile', 'China', 'Colombia',
       'Congo (Brazzaville)', 'Congo (Kinshasa)', 'Costa Rica',
       "Cote d'Ivoire", 'Croatia', 'Cuba', 'Cyprus', 'Czechia', 'Denmark',
       'Greenland', 'Djibouti', 'Dominican Republic', 'Ecuador', 'Egypt',
       'El Salvador', 'Equatorial Guinea', 'Eritrea', 'Estonia',
       'Eswatini', 'Ethiopia', 'Fiji', 'Finland', 'France', 'Gabon',
       'Gambia', 'Georgia', 'Germany', 'Ghana', 'Greece', 'Guatemala',
       'Guinea', 'Guyana', 'Haiti', 'Holy See', 'Honduras', 'Hungary',
       'Iceland', 'India', 'Ind

### Clean and explore the data

Since this table does not contain information for `Province/State` I'll remove the column

In [20]:
df = df.drop(columns=["Province/State"])
df.head()

,Country/Region,Lat,Long,Date,Confirmed,Deaths,Recovered,Active,WHO Region
0,Afghanistan,33.93911,67.709953,2020-01-22,0,0,0,0,Eastern Mediterranean
1,Albania,41.15330,20.168300,2020-01-22,0,0,0,0,Europe
2,Algeria,28.03390,1.659600,2020-01-22,0,0,0,0,Africa
3,Andorra,42.50630,1.521800,2020-01-22,0,0,0,0,Europe
4,Angola,-11.20270,17.873900,2020-01-22,0,0,0,0,Africa


Now lets explore the other columns and look for missing values

In [ ]:
# Take a look at the column types as well as null values
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 49068 entries, 0 to 49067
Data columns (total 9 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   Country/Region  49068 non-null  object 
 1   Lat             49068 non-null  float64
 2   Long            49068 non-null  float64
 3   Date            49068 non-null  object 
 4   Confirmed       49068 non-null  int64  
 5   Deaths          49068 non-null  int64  
 6   Recovered       49068 non-null  int64  
 7   Active          49068 non-null  int64  
 8   WHO Region      49068 non-null  object 
dtypes: float64(2), int64(4), object(3)
memory usage: 3.4+ MB


* There are no null objects in any of the remaining columns
* `Confirmed`, `Deaths`,  `Recovered`, and `Active` are all integers, which are good for mathematical analysis
* However, `Country/Region`, `Date`, and `Who Region` are all saved as objects, so I'll change them to more valuable types for data analysis

In [41]:
df['Date'] = pd.to_datetime(df['Date']) # Makes date into date format
df['Country/Region'] = df['Country/Region'].astype('category') # Makes Country into a category since we have multiple instances of each
df['WHO Region'] = df['WHO Region'].astype('category') # Makes Region a category since we have multiple instances of each

Now I'll use `sqlite3` to run SQL queries in Python

In [ ]:
import sqlite3, prettytable

# Include prettytable to display results
prettytable.DEFAULT = 'DEFAULT'

# Allow sql cell magic
%load_ext sql

# Make the database 'covid.db' with the table 'covid' and connect to it
con = sqlite3.connect("covid.db")
df.to_sql('covid', con, if_exists='replace', index=False)

# Use the database 'covid.db'
%sql sqlite:///covid.db

The sql extension is already loaded. To reload it, use:
  %reload_ext sql


In [30]:
%%sql
select *
from covid
limit 5;

 * sqlite:///covid.db
Done.


Country/Region,Lat,Long,Date,Confirmed,Deaths,Recovered,Active,WHO Region
Afghanistan,33.93911,67.709953,2020-01-22,0,0,0,0,Eastern Mediterranean
Albania,41.1533,20.1683,2020-01-22,0,0,0,0,Europe
Algeria,28.0339,1.6596,2020-01-22,0,0,0,0,Africa
Andorra,42.5063,1.5218,2020-01-22,0,0,0,0,Europe
Angola,-11.2027,17.8739,2020-01-22,0,0,0,0,Africa
